# Extract and display a synapse property matrix from a SONATA circuit

Copyright (c) 2025 Open Brain Institute

Authors: Christoph Pokorny

Last modified: 08.2025

## Summary
This analysis extracts and visualizes a matrix of a selected synapse property (conductance, delay, ...), grouped by a selected neuron property (layer, m-type, ...).
For details, see the [README](README.md).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time

from bluepysnap import Circuit
from connectome_manipulator.connectome_comparison import properties
from datetime import datetime
from entitysdk import Client, ProjectContext, models
from ipywidgets import widgets, interact
from obi_auth import get_token
from obi_notebook import get_projects
from obi_notebook import get_entities

## Circuit selection and download

A SONATA circuit will be analyzed. To download the data there, we must select 
the circuit, then download it. 

Should the circuit of interest already be placed on the local file system, you can skip ahead to the section `Circuit analysis` below.

#### Project selection
As a first step we select one of the projects we have access to that the circuit is associated with. If the circuit of interest is part of the public OBI assets, any project can be selected.

In [ ]:
token = get_token(environment="production", auth_mode="daf")
project_context = get_projects.get_projects(token)

In [ ]:
client = Client(environment="production", project_context=project_context, token_manager=token)

# Optional: Download using unique ID
entity_ID = "<CIRCUIT-ID>"  # <<< FILL IN UNIQUE CIRCUIT ID HERE


if entity_ID != "<CIRCUIT-ID>":
    circuit_ids = [entity_ID]
else:
# Alternative: Select from a table of entities
    circuit_ids = []
    circuit_ids = get_entities.get_entities("circuit", token, circuit_ids,
                                            project_context=project_context,
                                            multi_select=False,
                                            default_scale="small")

#### Fetch circuit
The circuit is copied to the local system at the expected location.

In [ ]:
# Fetch circuit
fetched = client.get_entity(entity_id=circuit_ids[0], entity_type=models.Circuit)
print(f"Circuit fetched: {fetched.name} (ID {fetched.id})\n")
print(f"#Neurons: {fetched.number_neurons}, #Synapses: {fetched.number_synapses}, #Connections: {fetched.number_connections}\n")
print(f"{fetched.description}\n")

# Download SONATA circuit files
asset = [asset for asset in fetched.assets if asset.label=="sonata_circuit"][0]
asset_dir = asset.path 
circuit_dir = "analysis_circuit_" + datetime.now().strftime('%Y-%m-%d_%H-%M-%S_%f')

t0 = time.time()
client.download_directory(
    entity_id=fetched.id,
    entity_type=models.Circuit,
    asset_id=asset.id,
    output_path=circuit_dir,
    max_concurrent=4,  # Parallel file download
)
t = time.time() - t0
print(f"Circuit files downloaded to '{os.path.join(circuit_dir, asset_dir)}' in {t:.1f}s")

## Circuit analysis

By default, the circuit from the download location is used. If a circuit from some other location us used, please modify the `circuit_config = ...` path below accordingly.

In [ ]:
# Path to existing circuit config
circuit_config = os.path.join(circuit_dir, asset_dir, "circuit_config.json")

assert os.path.exists(circuit_config), f"ERROR: Circuit config '{os.path.split(circuit_config)[1]}' not found!"

Loading SONATA circuit. Selections of the edge population containing the synapses, as well as pre-/post-synaptic node sets defining groups of neurons are possible.

In [ ]:
c = Circuit(circuit_config)
e_populations = c.edges.population_names
assert len(e_populations) > 0, "ERROR: No edge population found!"
node_sets = list(c.node_sets.content.keys())
e_popul_wdgt = widgets.Dropdown(options=e_populations, description="Edge population:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
pre_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Pre-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
post_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Post-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(e_popul_wdgt)
display(pre_nset_wdgt)
display(post_nset_wdgt)

## Grouping selection

The synaptic connectivity is grouped by one selected (categorical) neuron property, like layer, m-type, etc.

In [ ]:
def get_props(nodes):
    is_cat = nodes.get([]).dtypes.apply(lambda _x: isinstance(_x, pd.CategoricalDtype))
    props = is_cat[is_cat].index.to_list()  # Select categorical properties
    for _p in ["region", "layer", "mtype", "etype", "synapse_class"]:
        if _p in nodes.property_names:
            props = props + [_p]  # Add certain properties, even if not categorical
    return np.unique(props)
node_props = np.intersect1d(get_props(c.edges[e_popul_wdgt.value].source), get_props(c.edges[e_popul_wdgt.value].target))
groupby_wdgt = widgets.Dropdown(options=node_props, value="layer" if "layer" in node_props else node_props[0], description="Group-by:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(groupby_wdgt)

## Grouping function selection (optional)

A function for computing each matrix entry (i.e., a single value) from a group of values must be selected, such as mean, std (standard deviation), etc. By default, the mean is computed.

In [ ]:
fct_opt = [("Mean", "np.mean"), ("Std", "np.std"), ("Max", "np.max"), ("Min", "np.min"), ("Median", "np.median"), ("Sum", "np.sum")]
fct_wdgt = widgets.Dropdown(options=fct_opt, description="Grouping function:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(fct_wdgt)

## Synapse properties extraction

Extract the mean synapse property values for all pairs of groups using functionality from [connectome-manipulator](https://github.com/openbraininstitute/connectome-manipulator).

In [ ]:
props_dict = properties.compute(c, sel_src={"node_set": pre_nset_wdgt.value}, sel_dest={"node_set": post_nset_wdgt.value}, edges_popul_name=e_popul_wdgt.value, group_by=groupby_wdgt.value, fct=fct_wdgt.value)

## Interactive visualization

Interactive visualization of the synapse property matrix. The user can select what property to display. Also, empty groups (i.e., groups w/o any neurons or connections) can be excluded using a checkbox.

In [ ]:
# Interactive plot function
def plot_fct(res_sel, empty_sel):
    def filter_props_dict(props_dict, res_sel):
        out_dict = {}
        p = props_dict["conductance"]["data"]  # TODO: FIXME (better solution w/o using conductance hard-coded)
        pre_sel = np.any(p > 0.0, axis=1)
        post_sel = np.any(p > 0.0, axis=0)
        out_dict[res_sel] = props_dict[res_sel].copy()
        out_dict[res_sel]["data"] = out_dict[res_sel]["data"][pre_sel, :][:, post_sel]
        out_dict["common"] = props_dict["common"].copy()
        out_dict["common"]["src_group_values"] = [_v for _v, _s in zip(out_dict["common"]["src_group_values"], pre_sel) if _s]
        out_dict["common"]["tgt_group_values"] = [_v for _v, _s in zip(out_dict["common"]["tgt_group_values"], post_sel) if _s]
        return out_dict
    
    if empty_sel:
        plot_dict = props_dict
    else:
        plot_dict = filter_props_dict(props_dict, res_sel)

    if plot_dict[res_sel]["data"].size > 0:
        vmax = np.nanmax(plot_dict[res_sel]["data"])
        if vmax == 0.0:
            vmax = 1.0
        properties.plot(plot_dict[res_sel], plot_dict["common"], vmin=0.0, vmax=vmax, group_by=groupby_wdgt.value)
    else:
        plt.figure()
        plt.text(0, 0, "Nothing to show...")
        plt.axis("off")
        plt.axis("tight")
        plt.show()

In [ ]:
def prop_incl(prop):
    """Exclude certain (non-physiological) properties"""
    for _p in ["afferent_", "efferent_", "@", "_id"]:
        if _p in prop:
            return False
    return True
res_sel_opt = [(_v["unit"], _k) for _k, _v in sorted(props_dict.items()) if "unit" in _v and prop_incl(_k)]
res_sel_wdgt = widgets.Dropdown(options=res_sel_opt, description="Display:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
empty_sel_wdgt = widgets.Checkbox(value=True, description="Show empty groups", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
iplot = interact(plot_fct, res_sel=res_sel_wdgt, empty_sel=empty_sel_wdgt)